In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import unidecode
import os

In [2]:
soup = BeautifulSoup(open("../datasets/source_code_mutual_art.html"))
soup2 = BeautifulSoup(open("../datasets/artistas_seguidos_1.txt").read())
soup3 = BeautifulSoup(open("../datasets/artistas_seguidos_2.txt").read())
soup4 = BeautifulSoup(open("../datasets/artistas_seguidos_3.txt").read())
soup5 = BeautifulSoup(open("../datasets/artistas_seguidos_5.txt").read())
soup6 = BeautifulSoup(open("../datasets/artistas_seguidos_6.txt").read())
soup7 = BeautifulSoup(open("../datasets/artistas_seguidos_7.txt").read())
soup8 = BeautifulSoup(open("../datasets/paises_latam_1.txt").read())
soup9 = BeautifulSoup(open("../datasets/paises_latam_2.txt").read())
soup10 = BeautifulSoup(open("../datasets/paises_latam_3.txt").read())

soups = [soup,soup2,soup3,soup4,soup5,soup6,soup7,soup8,soup9,soup10]

In [3]:
precios_fecha=[['precio_estimado','precio_venta','fecha','autor','casa_subasta','tipo','tecnica','dimensiones','fecha_creacion','titulo']]
for soup in soups:
    obras = soup.findAll("div",{"class":"item table-row row v2__page"})
    for obra in obras:
        features=obra.findAll(lambda tag: tag.name=='div' and tag.get("class")==['v2__artwork-detail__price-col'])
        datos_obra=obra.findAll(lambda tag: tag.get("class")==['v2__link--primary'])
        tecnica = obra.findAll("span", {"itemprop": "artMedium"})
        dimensiones = obra.findAll("span", {"itemprop": "artDimensions"})
        fecha_creacion = obra.findAll("span", {"itemprop": "dateCreated"})
        titulo = obra.find("a", {"href": re.compile(r"/Artwork/.*")})
        if (len(features)==3 and len(datos_obra)==3 and len(tecnica)==1 and len(dimensiones)>=1):
            if (len(fecha_creacion) > 0):
                fecha_pintura = fecha_creacion[0].text
            else: 
                fecha_pintura = None
            precio_estimado=features[0].text
            precio_venta=features[1].text
            fecha=features[2].text
            autor=datos_obra[0].text
            casa_subasta=datos_obra[1].text
            tipo=datos_obra[2].text
            tecnica=tecnica[0].text
            dimensiones=dimensiones[0].text
            titulo = titulo.find("img")['title']
            precios_fecha.append([precio_estimado,precio_venta,fecha,autor,casa_subasta,tipo,tecnica,dimensiones,fecha_pintura,titulo])
        else:
            pass

In [4]:
len(precios_fecha)

22298

In [5]:
data=pd.DataFrame(precios_fecha[1:],columns=precios_fecha[0])
for column in data.columns:
    data[str(column)]=data[str(column)].replace(r'\s+', ' ', regex=True)
    data[str(column)]=data[unidecode.unidecode(str(column))]
data.drop_duplicates(inplace=True)

In [6]:
data.shape

(20128, 10)

In [9]:
data.tail(50)

,precio_estimado,precio_venta,fecha,autor,casa_subasta,tipo,tecnica,dimensiones,fecha_creacion,titulo
22239,"6,000 - 8,000 USD","7,500 USD","Feb 12, 2009",Daniel García,Sotheby's New York,Impressionist and Modern Art,"Oil on canvas, unframed",37 3/4 by 50 3/8 in. (alt: 96 by 128 cm.),None,Home Stories by Daniel García
22240,500 - 700 USD,225 USD,"Feb 13, 2009",Tómas Golding,Freeman's,Freemans Friday: Paintings and Prints,oil on panel,6 1/2 x 10 in,1922,"SAN BERNARDINO by Tómas Golding, 1922"
22241,500 - 700 USD,Not Sold,"Feb 13, 2009",Alceu Ribeiro,Freeman's,Freemans Friday: Paintings and Prints,oil on canvas,21 3/8 x 28 7/8 in,None,PUNTA GORDA by Alceu Ribeiro
22242,"3,500 - 4,500 USD","3,000 USD","Feb 19, 2009",Manuel Álvarez Bravo,Swann Auction Galleries,100 Fine Photographs,Platinum print on a golden paper,8 3/4x6 inches (22.2x15.2 cm.),Circa 1938; printed circa 1975,"""La Desvendada [Unbandaged]."" by Manuel Álvare..."
22243,"4,000 - 6,000 USD",Not Sold,"Feb 19, 2009",Manuel Álvarez Bravo,Swann Auction Galleries,100 Fine Photographs,Silver print,9 1/2x6 3/4 inches (24.1x17.1 cm.),1930s; printed 1970s,"""Señales y Pronósticos [Signs and Predictions]..."
22244,"10,000 - 15,000 USD","12,000 USD","Feb 19, 2009",Manuel Álvarez Bravo,Swann Auction Galleries,100 Fine Photographs,Silver print,8 1/2x6 1/2 inches (21.6x16.5 cm.),1944; printed 1950s-early 1960s,"""Un Pez que Llaman Sierra [A Fish Called Sword..."
22245,"4,000 - 6,000 USD","3,200 USD","Feb 19, 2009",Manuel Álvarez Bravo,Swann Auction Galleries,100 Fine Photographs,Oversize silver print,13 1/2x10 1/4 inches (34.3x26 cm.),1947; printed 1999,"""Espejo Negro [Black Mirror]."" by Manuel Álvar..."
22246,"15,000 - 20,000 USD","18,300 USD","Feb 22, 2009",Ángel Botello,"Hindman, Chicago","Fine Photographs, Prints and Works on Paper, 1...",oil on board,17 x 27 inches,None,"Untitled, Two Girls by Ángel Botello"
22247,"1,000 - 2,000 USD","1,037 USD","Feb 22, 2009",Gustavo Novoa,"Hindman, Chicago","Fine Photographs, Prints and Works on Paper, 1...",acrylic on board,14 1/2 x 20 1/2 inches,1973,"Magic Garden II by Gustavo Novoa, 1973"
22248,"1,000 - 2,000 USD",976 USD,"Feb 22, 2009",Gustavo Novoa,"Hindman, Chicago","Fine Photographs, Prints and Works on Paper, 1...",acrylic on board,14 1/2 x 21 inches,1973,"Magic Garden I by Gustavo Novoa, 1973"


In [10]:
data.to_csv('../datasets/data_mutualart_completo.csv')